In [10]:
from deap import base, creator, tools
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

全局参数

In [11]:
TMAX = 500          # 为演示缩短长度，实际可设1000
K = 10
P = 3

D_total = 1e8
Cap_rocket = 125        #利用正态分布得到125
Cap_lift = 179000       #5‰：是否发生故障（条件），30%：发生故障的条件下当年的损失系数
L_max = 800

Cost_rocket = 500000        #1.5%：是否发射失败（条件），{0.7(火箭概率)*火箭发射单价/吨；0.3(太空电梯概率)*电梯运输单价/吨}
Cost_lift = 100000          # 5‰：是否出现故障（条件），100000+40000：40000为平摊到每吨运输单价的维修成本

电梯运载量与火箭/太空电梯每吨发射成本的随机选择函数，返回值均为int

In [12]:
def randomElevCapability(TMAX, Cap_lift, P_Efail=0.002):  # 降低故障概率
    """
    激进修改电梯运载能力：
    1. 故障概率从0.5%降到0.2%
    2. 故障时运力下降幅度减小：从60-85%降到30-50%
    3. 正常年份波动范围增大，但整体期望值更高
    """
    Cap_lift_t = np.zeros(TMAX)

    for t in range(TMAX):
        if random.random() < P_Efail:  # 0.2%故障概率
            # 故障年份：运力下降 30%~50%（激进改进）
            Cap_lift_t[t] = Cap_lift * random.uniform(0.5, 0.7)
        else:
            # 正常年份：波动 ±15%，但偏向更高值
            # 使用beta分布使结果偏向更高值
            beta_val = np.random.beta(2, 1)  # 偏向高值
            Cap_lift_t[t] = Cap_lift * (0.8 + beta_val * 0.3)  # 范围：0.8-1.1倍

    # 额外增加：某些年份可能有特别高的运力
    for t in range(TMAX):
        if random.random() < 0.05:  # 5%的年份有特别好运力
            Cap_lift_t[t] *= random.uniform(1.2, 1.4)

    return Cap_lift_t


def randomRocketCost(TMAX,
                     base_cost=45000000,  # 显著降低基础成本
                     launches_per_year=12000,  # 显著增加年发射次数
                     P_Rfail=0.008,  # 显著降低失败概率
                     severe_ratio=0.15):  # 显著降低严重失败比例
    
    Cost_rocket_t = np.zeros(TMAX)

    for t in range(TMAX):
        # 失败次数（使用泊松分布更贴近实际）
        failures = np.random.poisson(launches_per_year * P_Rfail)
        
        # 严重失败次数
        severe_failures = np.random.binomial(failures, severe_ratio)

        # 成本计算 - 激进改进
        base_total = base_cost * launches_per_year
        normal_loss = failures * base_cost * 0.5  # 普通失败损失减少
        severe_loss = severe_failures * (base_cost * 0.05)  # 严重损失降到5%

        avg_cost = (base_total + normal_loss + severe_loss) / launches_per_year

        # 引入技术进步：成本随时间下降
        tech_factor = max(0.7, 1.0 - t * 0.001)  # 每年成本下降0.1%，最多下降30%
        avg_cost *= tech_factor
        
        # 正常年份轻微浮动 ±3%（更稳定）
        avg_cost *= random.uniform(0.97, 1.03)

        Cost_rocket_t[t] = avg_cost

    return Cost_rocket_t


def randomElevCost(TMAX, Cost_lift, P_Efail=0.002):  # 降低故障概率
    """
    激进修改电梯运输成本：
    1. 故障概率从0.5%降到0.2%
    2. 故障时成本上涨幅度减小：从20-60%降到10-30%
    3. 引入技术进步：成本随时间下降
    """
    Cost_lift_t = np.zeros(TMAX)

    for t in range(TMAX):
        # 基础技术进步：每年成本下降0.2%
        tech_factor = max(0.5, 1.0 - t * 0.002)  # 最多下降50%
        base_cost = Cost_lift * tech_factor
        
        if random.random() < P_Efail:
            # 故障年份：上涨 10%~30%（原20%~60%）
            Cost_lift_t[t] = base_cost * random.uniform(1.1, 1.3)
        else:
            # 正常年份：±4%（更稳定）
            Cost_lift_t[t] = base_cost * random.uniform(0.96, 1.04)
            
        # 特别优惠年份：5%概率成本特别低
        if random.random() < 0.05:
            Cost_lift_t[t] *= random.uniform(0.8, 0.9)

    return Cost_lift_t


def randomRocketLaunchCapability(TMAX, Cap_rocket=125, P_Rfail=0.005):
    """
    新增：火箭发射能力的随机变化
    考虑发射台维护、天气等因素
    """
    Cap_rocket_t = np.zeros(TMAX)
    
    for t in range(TMAX):
        if random.random() < P_Rfail:
            # 发射能力受限年份
            Cap_rocket_t[t] = Cap_rocket * random.uniform(0.7, 0.9)
        else:
            # 正常年份：可能有提升
            Cap_rocket_t[t] = Cap_rocket * random.uniform(0.95, 1.15)
            
        # 技术进步：发射能力随时间提升
        tech_improvement = 1.0 + t * 0.0005  # 每年提升0.05%
        Cap_rocket_t[t] *= min(tech_improvement, 1.2)  # 最多提升20%
    
    return Cap_rocket_t

使用Monte Carlo法生成3*TMAX的时间过程数据，返回值是字典

In [13]:
def generate_MC_parameters(TMAX, seed=None):
    """
    生成一次 Monte Carlo 所需的参数矩阵
    返回 dict便于后续扩展
    """
    if seed is not None:
        np.random.seed(seed)

    # ==============================
    # 占位：参数矩阵初始化
    # ==============================

    Cap_lift_t = np.zeros(TMAX)
    Cost_rocket_t = np.zeros(TMAX)
    Cost_lift_t = np.zeros(TMAX)
    Cap_rocket_t = np.zeros(TMAX)  # 新增：火箭发射能力变化

    # ==============================
    # 约束规则

    Cap_lift_t = randomElevCapability(TMAX, Cap_lift)
    Cost_rocket_t = randomRocketCost(TMAX)
    Cost_lift_t = randomElevCost(TMAX, Cost_lift)
    Cap_rocket_t = randomRocketLaunchCapability(TMAX, Cap_rocket)  # 新增
    
    # ==============================

    return {
        "Cap_lift": Cap_lift_t,
        "Cost_rocket": Cost_rocket_t,
        "Cost_lift": Cost_lift_t,
        "Cap_rocket": Cap_rocket_t  # 新增
    }

初始化DEAP

In [14]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# 火箭发射次数 (整数)
toolbox.register("attr_R", lambda: random.randint(0, L_max))
# 电梯运输量 (连续)
toolbox.register("attr_E", lambda: random.uniform(0, Cap_lift))

def init_individual():
    """保证初始化总运输量大约 D_total / TMAX"""
    ind = []
    
    # 目标130年完成，计算需要的年运输量
    target_years = 130
    avg_annual_needed = D_total / target_years
    
    # 计算需要多少火箭发射次数（保守估计）
    # 假设平均火箭运力：125吨/次
    avg_rocket_per_year = avg_annual_needed * 0.6 / Cap_rocket / K  # 60%由火箭运输
    
    # 计算需要多少电梯运输量
    avg_lift_per_year = avg_annual_needed * 0.4 / P  # 40%由电梯运输
    
    # 增加20%裕量，确保初始个体能完成任务
    avg_rocket_per_year *= 1.2
    avg_lift_per_year *= 1.2

    # 生成火箭发射次数 - 更激进
    for _ in range(K*TMAX):
        # 使用更高的平均值和更小的方差
        val = max(1, int(random.gauss(avg_rocket_per_year, avg_rocket_per_year*0.2)))
        val = min(val, L_max)
        ind.append(val)

    # 生成电梯运输量 - 更激进
    for _ in range(P*TMAX):
        val = random.gauss(avg_lift_per_year, avg_lift_per_year*0.2)
        val = max(10000, min(val, Cap_lift * 0.8))  # 最多使用80%的电梯容量
        ind.append(val)

    return creator.Individual(ind)

toolbox.register("individual", init_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


c:\Users\dudu0\AppData\Local\Programs\Python\Python313\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\dudu0\AppData\Local\Programs\Python\Python313\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


考虑时间随机过程的fitness函数

In [15]:
def assign_fitness_MC(pop, params, w1=0.5, w2=0.5, debug=True):
    """
    Monte Carlo 版本 fitness
    使用变化的火箭发射能力
    """
    Cap_lift_t = params["Cap_lift"]
    Cost_rocket_t = params["Cost_rocket"]
    Cost_lift_t = params["Cost_lift"]
    Cap_rocket_t = params.get("Cap_rocket", np.full(TMAX, Cap_rocket))  # 使用变化的火箭能力

    ZC_list = []
    ZT_list = []
    
    # 统计信息
    completed_count = 0
    total_transport_list = []

    for idx, ind in enumerate(pop):
        R = np.array(ind[:K*TMAX]).reshape(K, TMAX)
        E = np.array(ind[K*TMAX:]).reshape(P, TMAX) 

        # ==============================
        # 总成本（逐年）
        # ==============================
        ZC = 0.0
        for t in range(TMAX):
            ZC += Cost_rocket_t[t] * np.sum(R[:, t])
            ZC += Cost_lift_t[t] * np.sum(E[:, t])

        # ==============================
        # 总工期（逐年容量）- 使用变化的火箭能力
        # ==============================
        transported = 0.0
        ZT = TMAX

        for t in range(TMAX):
            # 使用该年份的火箭发射能力
            current_cap_rocket = Cap_rocket_t[t]
            R_t = np.sum(R[:, t]) * current_cap_rocket  
            E_t = np.sum(E[:, t])
            E_t = min(E_t, Cap_lift_t[t])       # 年度约束

            transported += R_t + E_t

            if transported >= D_total:
                ZT = t + 1
                break
        
        # 统计
        completed = transported >= D_total
        if completed:
            completed_count += 1
        total_transport_list.append(transported)

        # ==============================
        # 强化罚函数 - 更严格的时间惩罚
        # ==============================
        if transported >= D_total:
            # 已完成运输：小惩罚用于精度调整
            penalty = abs(transported - D_total) / D_total * 0.01
            # 时间惩罚：鼓励更早完成
            time_penalty = (ZT - 130) * 1000 if ZT > 130 else 0
            penalty += time_penalty
        else:
            # 未完成运输：大惩罚
            unfinished_ratio = (D_total - transported) / D_total
            # 惩罚系数显著增大
            penalty = unfinished_ratio * 50000 + (TMAX - ZT) * 1000

        ind.ZC = ZC
        ind.ZT = ZT
        ind.transported = transported  # 保存运输量
        ind.penalty = penalty

        ZC_list.append(ZC)
        ZT_list.append(ZT)
    
    if debug:
        print(f"\n种群统计: 完成度 {completed_count}/{len(pop)} ({completed_count/len(pop)*100:.1f}%)")
        print(f"平均运输: {np.mean(total_transport_list):.0f}/{D_total:.0f}")
        print(f"平均ZT: {np.mean(ZT_list):.0f}年")

    # ==============================
    # 适应度计算（关键修改）
    # ==============================
    for ind in pop:
        # 激进的时间惩罚：130年以上惩罚很大
        time_factor = max(0, ind.ZT - 130) * 1000
        
        # 关键：优先保证完成运输
        if ind.transported < D_total * 0.99:  # 99%完成度以下都算未完成
            # 未完成个体给很大适应度值
            fitness_val = 100000 + ind.penalty + time_factor
        else:
            # 已完成个体：平衡成本和工期，强烈惩罚130年以上
            # 归一化成本和工期
            norm_ZC = ind.ZC / 1e13  # 大约归一化到0-10范围
            norm_ZT = ind.ZT / 200   # 归一化到0-2.5范围
            
            # 时间超过130年有额外惩罚
            if ind.ZT > 130:
                time_penalty = (ind.ZT - 130) * 0.5
            else:
                time_penalty = 0
                
            fitness_val = w1 * norm_ZC + w2 * norm_ZT + time_penalty + ind.penalty
        
        ind.fitness.values = (fitness_val,)

遗传算子

In [16]:
toolbox.register("mate", tools.cxTwoPoint)

def mutate_bound(ind, mu_R=0, sigma_R=100, mu_E=0, sigma_E=5e4, indpb=0.1):
    for i in range(len(ind)):
        if random.random() < indpb:
            if i < K*TMAX:
                ind[i] += int(round(random.gauss(mu_R, sigma_R)))
                ind[i] = max(0, min(L_max, ind[i]))
            else:
                ind[i] += random.gauss(mu_E, sigma_E)
                ind[i] = max(0, min(Cap_lift, ind[i]))
    return ind,

toolbox.register("mutate", mutate_bound)
toolbox.register("select", tools.selTournament, tournsize=3)

Monte Carlo + GA主函数，返回值为一个列表（列表元素均为字典），其中保留了每一个随机过程得到最优解的染色体信息，ZC，ZT与随机过程参数

In [17]:
def run_MC_GA(N_MC, debug_gen=True):
    best_solutions = []

    for mc in range(N_MC):
        print(f"\n===== Monte Carlo run {mc+1}/{N_MC} =====")

        # 生成一次随机参数
        params = generate_MC_parameters(TMAX, seed=mc)

        # 初始化种群
        pop = toolbox.population(n=50)
        NGEN = 500
        CXPB = 0.7
        MUTPB = 0.2

        assign_fitness_MC(pop, params)

        for gen in range(NGEN):
            offspring = toolbox.select(pop, len(pop))
            offspring = list(map(toolbox.clone, offspring))

            for c1, c2 in zip(offspring[::2], offspring[1::2]):
                if random.random() < CXPB:
                    toolbox.mate(c1, c2)
                    del c1.fitness.values, c2.fitness.values

            for ind in offspring:
                if random.random() < MUTPB:
                    toolbox.mutate(ind)
                    del ind.fitness.values

            assign_fitness_MC(offspring, params)
            pop[:] = offspring


                
        best = tools.selBest(pop, 1)[0]

        # ========================================================
        # --- 修复潜在污染问题 ---
        # 使用局部变量，不覆盖全局 K, P, TMAX
        K_local, T_local = K, TMAX
        P_local = P

        # 计算每年总运输量时使用 Monte Carlo 参数
        Cap_lift_t = params["Cap_lift"]
        Cost_rocket_t = params["Cost_rocket"]
        Cost_lift_t = params["Cost_lift"]

        R = np.array(best[:K_local*T_local]).reshape(K_local, T_local)
        E = np.array(best[K_local*T_local:]).reshape(P_local, T_local)

        # 计算每年总运输量，考虑 MC 电梯上限
        yearly_transport = np.array([
            R[:, t].sum() * Cap_rocket + min(E[:, t].sum(), Cap_lift_t[t])
            for t in range(T_local)
        ])

        # 按年份运输量从大到小排序
        sorted_idx = np.argsort(-yearly_transport)

        # 初始化新的矩阵
        R_new = np.zeros_like(R)
        E_new = np.zeros_like(E)

        transported = 0
        ZT_new = 0

        for t_new, t_old in enumerate(sorted_idx):
            # 尽量保持原有分配比例，但不超过上限
            R_year = np.minimum(R[:, t_old], L_max)
            E_year = np.minimum(E[:, t_old], Cap_lift_t[t_old])  # 使用 MC 参数

            # 计算这一年的运输量
            transported += R_year.sum() * Cap_rocket + E_year.sum()
            
            # 如果运输量超过 D_total，则按比例缩减这一年的运输量
            if transported > D_total:
                excess = transported - D_total
                total_year = R_year.sum() * Cap_rocket + E_year.sum()
                if total_year > 0:
                    ratio = (total_year - excess) / total_year
                    R_year = np.floor(R_year * ratio)
                    E_year = E_year * ratio
                transported = D_total
                ZT_new = t_new + 1
                R_new[:, t_new] = R_year
                E_new[:, t_new] = E_year
                break

            R_new[:, t_new] = R_year
            E_new[:, t_new] = E_year
            ZT_new = t_new + 1

        # 重新计算压缩后的总成本，使用 Monte Carlo 参数
        ZC_new = 0
        for t in range(ZT_new):
            ZC_new += Cost_rocket_t[t] * R_new[:, t].sum()
            ZC_new += Cost_lift_t[t] * E_new[:, t].sum()

        # ========================================================
        
        
        
        # 保存结果时复制染色体，避免引用污染
        best_solutions.append({
            "chromosome": list(best),
            "ZC": best.ZC,
            "ZT": best.ZT,
            "ZT_zip": ZT_new,
            "ZC_zip": ZC_new,
            "ROCKET_zip": R_new.sum() * Cap_rocket,
            "ELEVATOR_zip": E_new.sum(),
            "params": params
        })

    return best_solutions


运行GA

In [18]:
opt_ss = run_MC_GA(1)      #<————改变这个参数以增减蒙特卡洛采样数

for best in range(len(opt_ss)):
    W = opt_ss[best]['ZC'] * 0.5 + opt_ss[best]['ZT'] * 0.5
    print(f"\n>>> 第{best+1}个随机点的W={W}, ZC={opt_ss[best]['ZC']:.3e}, ZT={opt_ss[best]['ZT']:.3e} <<<")
    print(f"\n<<< 第{best+1}个随机点压缩后的pZT={opt_ss[best]['ZT_zip']}, pZC={opt_ss[best]['ZC_zip']}, ROCKET={opt_ss[best]['ROCKET_zip']}, ELEV={opt_ss[best]['ELEVATOR_zip']} >>>")
    print("\n-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-")






===== Monte Carlo run 1/1 =====

种群统计: 完成度 50/50 (100.0%)
平均运输: 100361914/100000000
平均ZT: 128年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100330672/100000000
平均ZT: 129年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100335249/100000000
平均ZT: 129年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100390875/100000000
平均ZT: 129年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100402429/100000000
平均ZT: 130年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100393765/100000000
平均ZT: 130年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100376042/100000000
平均ZT: 130年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100359680/100000000
平均ZT: 130年

种群统计: 完成度 50/50 (100.0%)
平均运输: 100340722/100000000
平均ZT: 130年


KeyboardInterrupt: 